In [8]:
import stanza
stanza.download('en')

nlp_stanza = stanza.Pipeline('en',processors= 'tokenize,ner')
nlp_stanza.processors['ner'].get_known_tags()

2024-06-09 20:01:12 INFO: Downloaded file to C:\Users\alber\stanza_resources\resources.json
2024-06-09 20:01:12 INFO: Downloading default packages for language: en (English) ...
2024-06-09 20:01:13 INFO: File exists: C:\Users\alber\stanza_resources\en\default.zip
2024-06-09 20:01:16 INFO: Finished downloading models and saved to C:\Users\alber\stanza_resources
2024-06-09 20:01:16 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-09 20:01:16 INFO: Downloaded file to C:\Users\alber\stanza_resources\resources.json
2024-06-09 20:01:16 WARNING: Language en package default expects mwt, which has been added
2024-06-09 20:01:16 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                 

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [9]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

## Comparison

In [10]:
import numpy as np
import pandas as pd

Loading the dfs

In [13]:
df_entities_spacy = pd.read_csv("df_entities_spacy.csv")

In [14]:
df_entities_stanza = pd.read_csv("df_entities_stanza.csv")

In [15]:
list_people = np.unique(df_entities_spacy["subject"])

Hacking the subject stuff

In [16]:
df_entities_stanza["subject"] = df_entities_stanza["source_index"]

In [17]:
df_no_duplicates = df_entities_spacy[["subject", "source_index"]]

In [18]:
df_no_duplicates = df_no_duplicates.drop_duplicates()
df_no_duplicates.reset_index(inplace=True, drop=True)

In [19]:
list_subjects = df_no_duplicates["subject"].to_list()
list_indexes = df_no_duplicates["source_index"].to_list()

In [20]:
translation_dict = {}
for index in list_indexes:
    translation_dict[index] = list_subjects[index]

In [21]:
df_entities_stanza["subject"] = df_entities_stanza["source_index"].replace(translation_dict)

### FX

ISSUE WITH DUPLICATES, GO THROUGH ROWS INSTEAD

In [22]:
aggregate_results = []

for people in list_people:
    
    results = {}
    results["subject"] = people
    
    filter_df_spacy = df_entities_spacy[df_entities_spacy["subject"] == people][["text", "label", "start_pos", "end_pos"]]
    filter_df_stanza = df_entities_stanza[df_entities_stanza["subject"] == people][["text", "label", "start_pos", "end_pos"]]
    
    # print(filter_df_spacy.head())
    # print(filter_df_stanza.head())
    
    entities_spacy = filter_df_spacy["text"].to_list()
    # print(entities_spacy)
    # print("n_spacy:", len(entities_spacy))
    
    entities_stanza = filter_df_stanza["text"].to_list()
    # print(entities_stanza)
    # print("n_stanza:",len(entities_stanza))
    
    total_entities = entities_spacy + entities_stanza
    
    
    labels_spacy = filter_df_spacy["label"].to_list()
    labels_stanza = filter_df_stanza["label"].to_list()
    
    print(labels_spacy)
    print(labels_stanza)
    
    
    dict_label_spacy = {k:v for k in entities_spacy for v in labels_spacy}
    dict_label_stanza = {k:v for k in entities_stanza for v in labels_stanza}    
    
    # Entity agreements
    common_entities = [ent for ent in total_entities if ent in entities_stanza and ent in entities_spacy]
    print(common_entities)
    print(len(common_entities))
    
    
    spacy_diff = [ent for ent in total_entities if ent not in entities_stanza and ent in entities_spacy]
    # print(spacy_diff)
    stanza_diff =  [ent for ent in total_entities if ent not in entities_spacy and ent in entities_stanza]
    # print(stanza_diff)
    
    total_num_entities = len(common_entities) + len(spacy_diff) + len(stanza_diff)
    diff_spacy = len(spacy_diff)/total_num_entities
    diff_stanza = len(stanza_diff)/total_num_entities
    
    total_num_disagreements = len(spacy_diff) + len(stanza_diff)
    
    agreement_entities = len(common_entities)/total_num_entities
    
    results["agreement_total"] = len(common_entities)
    results["agreement_ratio"] = agreement_entities
    results["diff_spacy_entities"] = len(spacy_diff)
    results["diff_stanza_entities"] = len(stanza_diff)
    
    #Partial Span agreement
    
    if agreement_entities != 1.0: # if the agreement is not total - avoid zero division
        
        print("skip for now")
        
        # Issue with getting the spans for the entities:
        # sometimes the entity appears more than once so we can't get the index by filtering like filter[filter["text"] = ent] 
        # since it will return more than one value
        # ideally, a way to get each entity with its index to do filter[index][start] and filter[index][stop] instead
    
        # spacy_spans = []
        # for entity in spacy_diff:
        #     start = filter_df_spacy["start_pos"]
        #     stop = filter_df_spacy["end_pos"]
        #     print(start, stop)
        #     entity_spans = np.arange(start, stop + 1, 1) # CHECK IF THE STOP SPAN IN SPACY IS INCLUDING OR EXCLUDING
        #     spacy_spans.append(entity_spans)

        # stanza_spans = []
        # for entity in stanza_diff:
        #     start = filter_df_stanza["start_pos"]
        #     stop = filter_df_stanza["end_pos"]
        #     print(start, stop)
        #     entity_spans = np.arange(start, stop + 1, 1) # CHECK IF THE STOP SPAN IN STANZA IS INCLUDING OR EXCLUDING
        #     stanza_spans.append(entity_spans)

        # common_entity_spans = []
        # for entity_span_spacy in spacy_spans:
        #     for entity_span_stanza in stanza_spans:
        #         common_span = [index for index in entity_span_spacy if index in entity_span_stanza]
        #         if len(common_span) > 1:
        #             n_entity_spacy = spacy_spans.index(entity_span_spacy)
        #             n_entity_stanza = stanza_spans.index(entity_span_stanza)
        #             tuple_indexes = (n_entity_spacy, n_entity_stanza)
        #             common_entity_spans.append(tuple_indexes)
        
        # common_entity_spans = set(common_entity_spans)
        # partial_agreement_ratio = len(common_entity_spans)/total_num_disagreements
        # results["partial_agreements_over_total_disagreements"] = partial_agreement_ratio
        
        # total_partial_agreements = (len(common_entity_spans) + len(common_entities))/total_num_entities
        # results["partial_agreements"] = total_partial_agreements
    
    # Label agreements
    
    
    # print(dict_label_spacy)
    # print(dict_label_stanza)
    print([(ent, dict_label_spacy[ent]) for ent in common_entities])
    print([(ent, dict_label_stanza[ent]) for ent in common_entities])
    
    
    common_labels = [(ent, dict_label_spacy[ent]) for ent in common_entities if dict_label_spacy[ent] == dict_label_stanza[ent]]
    spacy_label_diff = [(ent, dict_label_spacy[ent]) for ent in common_entities if dict_label_spacy[ent] != dict_label_stanza[ent]]
    stanza_label_diff =  [(ent, dict_label_stanza[ent]) for ent in common_entities if dict_label_spacy[ent] != dict_label_stanza[ent]]
    
    if len(common_labels) > 0:
        agreement_labels = len(common_labels)/len(common_entities)
    else:
        agreement_labels = 0
    
    results["agreement_labels_total"] = len(common_labels)
    results["agreement_labels_ratio"] = agreement_labels
    
    
    if len(common_entities) == len(entities_stanza):
        agreement = "Total Agreement"
        
    else: 
        agreement = "Partial Agreement"
    
    aggregate_results.append(results)
    
    ############# LETF TO DO: COMPUTE STATS OF ACCURACY ETC


['NORP', 'DATE', 'DATE', 'NORP', 'WORK_OF_ART', 'DATE', 'PERSON', 'PERSON', 'PERSON', 'ORG', 'PERSON', 'ORG', 'PERSON', 'DATE', 'CARDINAL', 'WORK_OF_ART', 'CARDINAL', 'CARDINAL', 'WORK_OF_ART', 'PERSON', 'ORG', 'DATE', 'ORDINAL', 'CARDINAL', 'PERSON', 'PERSON', 'NORP', 'DATE', 'GPE', 'ORG', 'DATE', 'PERSON', 'PERSON', 'PERSON', 'GPE', 'GPE', 'ORG', 'ORG', 'ORG', 'PERSON', 'PERSON', 'PERSON', 'PERSON', 'PERSON', 'DATE', 'ORG', 'PERSON', 'GPE', 'PRODUCT', 'PERSON', 'FAC', 'ORG', 'DATE', 'NORP', 'GPE', 'DATE', 'ORG', 'DATE', 'NORP', 'NORP', 'PERSON', 'PERSON', 'NORP', 'NORP', 'GPE', 'PERSON', 'DATE', 'GPE', 'ORG', 'ORG', 'NORP', 'GPE', 'NORP', 'TIME', 'GPE', 'GPE', 'GPE', 'DATE', 'ORG', 'CARDINAL', 'PERSON', 'NORP', 'DATE', 'ORDINAL', 'GPE', 'ORG', 'PERSON', 'PERSON', 'PERSON', 'DATE', 'ORG', 'PERSON', 'PERSON', 'GPE', 'GPE', 'DATE', 'DATE', 'PERSON', 'GPE', 'DATE', 'DATE', 'PERSON', 'ORG', 'GPE', 'GPE', 'ORG', 'PERSON', 'GPE', 'ORG', 'DATE', 'GPE', 'GPE', 'DATE', 'CARDINAL', 'GPE', 'ORG'

In [23]:
aggregate_results

[{'subject': 'Aage Bohr',
  'agreement_total': 50,
  'agreement_ratio': 0.11261261261261261,
  'diff_spacy_entities': 264,
  'diff_stanza_entities': 130,
  'agreement_labels_total': 0,
  'agreement_labels_ratio': 0},
 {'subject': 'Aaron Klug',
  'agreement_total': 34,
  'agreement_ratio': 0.0947075208913649,
  'diff_spacy_entities': 134,
  'diff_stanza_entities': 191,
  'agreement_labels_total': 0,
  'agreement_labels_ratio': 0},
 {'subject': 'Adolf Friedrich Johann Butenandt',
  'agreement_total': 34,
  'agreement_ratio': 0.11447811447811448,
  'diff_spacy_entities': 160,
  'diff_stanza_entities': 103,
  'agreement_labels_total': 0,
  'agreement_labels_ratio': 0},
 {'subject': 'Adolf Otto Reinhold Windaus',
  'agreement_total': 25,
  'agreement_ratio': 0.13736263736263737,
  'diff_spacy_entities': 77,
  'diff_stanza_entities': 80,
  'agreement_labels_total': 0,
  'agreement_labels_ratio': 0},
 {'subject': 'Adolf von Baeyer',
  'agreement_total': 41,
  'agreement_ratio': 0.120943952802

INTER ANNOTATOR AGREEMENT BETWEEN THE TWO PACKAGES